<a href="https://colab.research.google.com/github/tabang205/Khai_Pha_Du_Lieu/blob/main/App_KPDL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%writefile app.py
import streamlit as st
import joblib
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Load mô hình đã lưu
model = joblib.load('car_model_rf.pkl')

# 2. Cấu hình trang
st.set_page_config(page_title="Dự đoán Mua Xe Oto", layout="wide")
st.title("🚗 Hệ thống Dự đoán Khách hàng Mua Ô tô")
st.markdown("Dựa trên thuật toán **Random Forest** (Độ chính xác ~97%)")

# 3. Sidebar: Nhập liệu
st.sidebar.header("Thông tin xe")

def user_input_features():
    # Mapping lại từ chữ sang số (Phải khớp với lúc huấn luyện)
    # Buying & Maint
    price_map = {'Thấp (Low)': 0, 'Trung bình (Med)': 1, 'Cao (High)': 2, 'Rất cao (V-High)': 3}
    buying = st.sidebar.selectbox('Giá mua xe', list(price_map.keys()))
    maint = st.sidebar.selectbox('Chi phí bảo dưỡng', list(price_map.keys()))

    # Doors
    doors_map = {'2': 2, '3': 3, '4': 4, '5 trở lên': 5}
    doors = st.sidebar.selectbox('Số cửa', list(doors_map.keys()))

    # Persons
    persons_map = {'2': 2, '4': 4, 'Nhiều hơn (More)': 5}
    persons = st.sidebar.selectbox('Số chỗ ngồi', list(persons_map.keys()))

    # Lug_boot
    boot_map = {'Nhỏ (Small)': 0, 'Vừa (Med)': 1, 'Lớn (Big)': 2}
    lug_boot = st.sidebar.selectbox('Kích thước cốp', list(boot_map.keys()))

    # Safety
    safe_map = {'Thấp (Low)': 0, 'Trung bình (Med)': 1, 'Cao (High)': 2}
    safety = st.sidebar.selectbox('Độ an toàn', list(safe_map.keys()))

    # Tổng hợp thành DataFrame
    data = {
        'buying': price_map[buying],
        'maint': price_map[maint],
        'doors': doors_map[doors],
        'persons': persons_map[persons],
        'lug_boot': boot_map[lug_boot],
        'safety': safe_map[safety]
    }
    return pd.DataFrame(data, index=[0])

input_df = user_input_features()

# 4. Hiển thị thông tin đã chọn
st.subheader("Thông số xe bạn chọn:")
st.table(input_df)

# 5. Nút Dự đoán
if st.button("🔍 Dự đoán ngay"):
    prediction = model.predict(input_df)

    # Mapping ngược lại từ số ra chữ để hiển thị kết quả
    result_map = {0: "KHÔNG NÊN MUA ❌", 1: "CÂN NHẮC 🤔", 2: "NÊN MUA NGAY ✅"}
    result_text = result_map[prediction[0]]

    if prediction[0] == 2:
        st.success(f"Kết quả: {result_text}")
        st.balloons() # Hiệu ứng bóng bay
    elif prediction[0] == 1:
        st.warning(f"Kết quả: {result_text}")
    else:
        st.error(f"Kết quả: {result_text}")

# 6. (Tùy chọn) Thêm tab Biểu đồ phân tích nếu muốn
st.divider()
st.info("Hệ thống được phát triển cho bài tập lớn môn KPDL.")

Writing app.py


In [ ]:
!curl ipv4.icanhazip.com

34.12.114.71


In [ ]:
!pip install pyngrok

from pyngrok import ngrok

# Dán Authtoken của bạn vào đây (QUAN TRỌNG)
# Ví dụ: ngrok.set_auth_token("2KvHb...")
ngrok.set_auth_token("36ywysaWvIQxgAgCrq2mk80vf8q_2t4mM7q16d4sQF8bDsUgv")

# hạy Streamlit dưới nền (Background process)
# nohup giúp code chạy ngầm mà không bị treo ô code này
!nohup streamlit run app.py --server.port 8501 &

# Mở kết nối ngrok tới cổng 8501
url = ngrok.connect(8501).public_url
print('🚀 Link App xịn sò của bạn là:', url)

nohup: appending output to 'nohup.out'
🚀 Link App xịn sò của bạn là: https://polymorphistic-unshaking-stacee.ngrok-free.dev
